In [1]:
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 1.8 MB/s eta 0:00:0000:0100:01


In [2]:
!pip install PeptideBuilder

In [3]:
!pip install parmed

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 1.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for parmed: filename=ParmEd-3.4.3-cp37-cp37m-linux_x86_64.whl size=835168 sha256=85a27aa9560ae4b951d6617498393669cbdec0cdd1d016911dbb7e2032ba2faf
  Stored in directory: /home/jovyan/.cache/pip/wheels/2f/26/b3/8cb8da47601e3057598009e903ba5d71e3a8ff08bcbc65cd1e
Successfully built parmed


In [4]:
import warnings
warnings.filterwarnings('ignore')
import nglview
from PeptideBuilder import Geometry
import PeptideBuilder
import Bio.PDB
import pymol
from Bio.PDB import *

In [5]:
def build_peptide(aa_str, add_oxt=True, add_hyd=False):
    aa_str = str(aa_str)
    
    if len(aa_str) < 1:
        return None
    
    geo = Geometry.geometry(aa_str[0])
    structure = PeptideBuilder.initialize_res(geo)
    
    for aa in aa_str[1:]:
        geo = Geometry.geometry(aa)
        PeptideBuilder.add_residue(structure, geo)
    
    if add_oxt:
        PeptideBuilder.add_terminal_OXT(structure)
    
    out = Bio.PDB.PDBIO()
    out.set_structure(structure)
    out.save("temp1.pdb")
    
    # Add hydrogens with pymol
    if add_hyd:
        pymol.cmd.load('temp1.pdb', aa_str)
        pymol.cmd.h_add()
        pymol.cmd.save('temp1.pdb')
        pymol.cmd.remove('all')
    
        parser = Bio.PDB.PDBParser()
        structure = parser.get_structure("structure", "temp1.pdb")
        
    return structure
        

In [6]:
structure = build_peptide("A"*10)

In [7]:
view = nglview.show_biopython(structure)
view.add_ball_and_stick()
view

NGLWidget()

# Simulation

In [8]:
i = 0

In [9]:
import parmed
import matplotlib.pyplot as plt
import mdtraj
import nglview
import numpy as np
from openmm import *
from openmm.app import *
from openmm.unit import *

def simulation(i):
    from sys import stdout


    pdb = PDBFile('temp1.pdb')
    modeller = Modeller(pdb.topology, pdb.positions)
    forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
    modeller.addHydrogens(forcefield)
    modeller.addSolvent(forcefield, model='tip3p', padding=0.3*nanometer)
    modeller.deleteWater()
    print(modeller.topology)

    # Write a PDB file to provide a topology of the solvated
    # system to MDTraj below.
    with open(f'init{i}.pdb', 'w') as outfile:
        PDBFile.writeFile(modeller.topology, modeller.positions, outfile)

    # The modeller builds a periodic box with the solute and solvent molecules.
    # PME is the method to compute long-range electristatic interactions in
    # periodic systems.
    system = forcefield.createSystem(
        modeller.topology, nonbondedMethod=PME, constraints=HBonds)
    temperature = 300 * kelvin
    pressure = 1 * bar
    integrator = LangevinIntegrator(temperature, 1/picosecond, 2*femtoseconds)
    system.addForce(MonteCarloBarostat(pressure, temperature))
    simulation = Simulation(modeller.topology, system, integrator)
    simulation.context.setPositions(modeller.positions)
    simulation.minimizeEnergy()
    simulation.reporters.append(DCDReporter(f'traj{i}.dcd', 100))

    #simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
    #                                              temperature=True, elapsedTime=True))
    simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
            potentialEnergy=True, temperature=True))
    #simulation.reporters.append(parmed.openmm.StateDataReporter(stdout, 1000, step=True,
    #                                              temperature=False))

    simulation.reporters.append(StateDataReporter(f"scalars{i}.csv", 1000, time=True, kineticEnergy=True,
                                                  potentialEnergy=True, totalEnergy=True, temperature=True))
    simulation.step(10000)

    # The last line is only needed for Windows users,
    # to close the DCD file before it can be opened by nglview.
    del simulation

In [10]:
simulation(i)

<Topology; 1 chains, 10 residues, 103 atoms, 102 bonds>
#"Step","Potential Energy (kJ/mole)","Temperature (K)"
1000,396.97948977580427,262.0762585452668
2000,445.1541687917975,285.24317159704214
3000,412.3607746823732,320.8937054261033
4000,292.6674952769099,312.922761274518
5000,123.09094405675523,317.8596545473115
6000,84.67981187740816,315.6065472653644
7000,130.88666831112005,317.5891402965934
8000,117.34812358035924,356.3217776998018
9000,118.16266439152469,299.6773104777109
10000,110.60913463462566,307.60921660042266


In [11]:
 i+=1

In [12]:
traj = mdtraj.load(f'traj{i-1}.dcd', top=f'init{i-1}.pdb')
view = nglview.show_mdtraj(traj)
view.clear_representations()
view.add_licorice()
view

NGLWidget(max_frame=99)

In [13]:
import pandas as pd
df = pd.read_csv(f"scalars{i-1}.csv")
df.head()

,"#""Time (ps)""",Potential Energy (kJ/mole),Kinetic Energy (kJ/mole),Total Energy (kJ/mole),Temperature (K)
0,2.0,396.979490,276.735953,673.715443,262.076259
1,4.0,445.154169,301.198748,746.352917,285.243172
2,6.0,412.360775,338.843457,751.204232,320.893705
3,8.0,292.667495,330.426644,623.094140,312.922761
4,10.0,123.090944,335.639691,458.730635,317.859655


## Energy

In [42]:
from openmm.app import *
from openmm import *
from openmm.unit import *

pdb = PDBFile('temp1.pdb')
forcefield = ForceField('amber14-all.xml')

modeller = Modeller(pdb.topology, pdb.positions)
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
modeller.addHydrogens(forcefield)
modeller.addSolvent(forcefield, model='tip3p', padding=0.3*nanometer)
modeller.deleteWater()

system = forcefield.createSystem(modeller.topology)

In [43]:
for i, f in enumerate(system.getForces()):
    print(i, f)
    f.setForceGroup(i)

0 <openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x7f47d4d6efc0> >
1 <openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x7f47d4d6eb70> >
2 <openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x7f47d4d6ee10> >
3 <openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x7f47d4d6eab0> >
4 <openmm.openmm.CMMotionRemover; proxy of <Swig Object of type 'OpenMM::CMMotionRemover *' at 0x7f47d4d6eba0> >


In [44]:
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.004*picoseconds)
simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)

In [45]:
for i, f in enumerate(system.getForces()):
    state = simulation.context.getState(getEnergy=True, groups={i})
    print(f.getName(), state.getPotentialEnergy())

HarmonicBondForce 5.0138750076293945 kJ/mol
HarmonicAngleForce 19.22382354736328 kJ/mol
NonbondedForce 22.4150390625 kJ/mol
PeriodicTorsionForce 284.3539733886719 kJ/mol
CMMotionRemover 0.0 kJ/mol


In [41]:
energy = simulation.context.getState(getEnergy=True, groups={1,2,3}).getPotentialEnergy()
energy

Quantity(value=325.747314453125, unit=kilojoule/mole)

In [24]:
#model = modify(structure, 5, "psi", 20)

## DR

In [15]:
def modify(structure, residue, diangle, angle_value):
    models = Selection.unfold_entities(structure, "M")
    model = models[0]
    model.atom_to_internal_coordinates()
    
    residues = Selection.unfold_entities(structure, "R")
    residue = residues[residue]
    
    diangle = residue.internal_coord.pick_angle(diangle)
    residue.internal_coord.set_angle(diangle, angle_value)
    model.internal_to_atom_coordinates()
    
    return model

In [ ]:
view = nglview.show_mdtraj(structure)
view.clear_representations()
view.add_licorice()
view